In [28]:
import pandas as pd
import matplotlib.pyplot as plt
# Create a discussion file path list
discussion_file1_path = "../Week1/DevGPT/snapshot_20230727/20230727_195954_discussion_sharings.json"
discussion_file2_path = "../Week1/DevGPT/snapshot_20230803/20230803_094811_discussion_sharings.json"
discussion_file3_path = "../Week1/DevGPT/snapshot_20230810/20230810_124048_discussion_sharings.json"
discussion_file4_path = "../Week1/DevGPT/snapshot_20230817/20230817_130721_discussion_sharings.json"
discussion_file5_path = "../Week1/DevGPT/snapshot_20230824/20230824_102000_discussion_sharings.json"
discussion_file6_path = "../Week1/DevGPT/snapshot_20230831/20230831_061926_discussion_sharings.json"

discussion_json_files = [discussion_file1_path,discussion_file2_path,discussion_file3_path,discussion_file4_path,discussion_file5_path,discussion_file6_path]

In [29]:
# Create an issue file path list
issue_file1_path = "../Week1/DevGPT/snapshot_20230727/20230727_195941_issue_sharings.json"
issue_file2_path = "../Week1/DevGPT/snapshot_20230803/20230803_094705_issue_sharings.json"
issue_file3_path = "../Week1/DevGPT/snapshot_20230810/20230810_123938_issue_sharings.json"
issue_file4_path = "../Week1/DevGPT/snapshot_20230817/20230817_130502_issue_sharings.json"
issue_file5_path = "../Week1/DevGPT/snapshot_20230824/20230824_101836_issue_sharings.json"
issue_file6_path = "../Week1/DevGPT/snapshot_20230831/20230831_061759_issue_sharings.json"

issue_json_files = [issue_file1_path,issue_file2_path,issue_file3_path,issue_file4_path,issue_file5_path,issue_file6_path]

In [30]:
# Concatenate two types of files
json_files = discussion_json_files + issue_json_files
json_files

['../Week1/DevGPT/snapshot_20230727/20230727_195954_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230803/20230803_094811_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230810/20230810_124048_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230817/20230817_130721_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230824/20230824_102000_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230831/20230831_061926_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230727/20230727_195941_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230803/20230803_094705_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230810/20230810_123938_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230817/20230817_130502_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230824/20230824_101836_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230831/20230831_061759_issue_sharings.json']

In [31]:
# Load the data into DataFrame from all files 
dataframes = []
for file in json_files:
    try:
        # load json files
        df = pd.read_json(file)
        dataframes.append(df["Sources"])
    except ValueError as e:
        print(f"error from {file} : {e}")

# combine all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df.info())
print(combined_df.head())

<class 'pandas.core.series.Series'>
RangeIndex: 1998 entries, 0 to 1997
Series name: Sources
Non-Null Count  Dtype 
--------------  ----- 
1998 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB
None
0    {'Type': 'discussion', 'URL': 'https://github....
1    {'Type': 'discussion', 'URL': 'https://github....
2    {'Type': 'discussion', 'URL': 'https://github....
3    {'Type': 'discussion', 'URL': 'https://github....
4    {'Type': 'discussion', 'URL': 'https://github....
Name: Sources, dtype: object


### 2. Can we identify patterns in the prompts developers use when interacting with ChatGPT, and do these patterns correlate with the success of issue resolution?

In [32]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import altair as alt

# Initialize lists to collect extracted data
Type_list = []
Number_list = []
Conversations_num_list = []
PromptLength_list = []
ResponseLength_list = []
FirstWord_list = []
PromptText_list = []  # Store prompts for later TF-IDF analysis
ClosedAt_list = []
State_list = []

# Iterate through the rows of the DataFrame
for i in range(len(combined_df)):
    row = combined_df.iloc[i]
    type_value = row.get("Type", "unknown")
    number = row.get("Number", 0)
    closed_at = row.get("ClosedAt", "unknown")
    state = row.get("State", "unknown")
    
    ChatgptSharing = row.get("ChatgptSharing", [])
    total_conversations = 0  # Counter for the number of conversations in a record
    
    for item in ChatgptSharing:
        Conversations = item.get("Conversations", [])
        total_conversations += len(Conversations)
        
        for conv in Conversations:
            prompt = conv.get("Prompt", "")
            answer = conv.get("Answer", "")
            
            # Token count using split()
            prompt_length = len(prompt)
            response_length = len(answer)
            
            # Extract first word of the prompt
            first_word = prompt.split()[0].lower() if prompt.split() else "unknown"
            
            # Append data
            Type_list.append(type_value)
            Number_list.append(number)
            Conversations_num_list.append(total_conversations)
            PromptLength_list.append(prompt_length)
            ResponseLength_list.append(response_length)
            FirstWord_list.append(first_word)
            PromptText_list.append(prompt)
            ClosedAt_list.append(closed_at)
            State_list.append(state)

# Create DataFrame
extracted_data = {
    "Type": Type_list,
    "Number": Number_list,
    "PromptLength": PromptLength_list,
    "ResponseLength": ResponseLength_list,
    "FirstWord": FirstWord_list,
    "PromptText": PromptText_list,
    "ClosedAt": ClosedAt_list,
    "State": State_list
}
df = pd.DataFrame(extracted_data)

# Count first word occurrences
first_word_counts = df["FirstWord"].value_counts()

# Convert first word counts to DataFrame for visualization
first_word_df = pd.DataFrame({"Start Word": first_word_counts.index, "Counts": first_word_counts.values})

# Create bar chart using Altair for start words
start_word_chart = alt.Chart(first_word_df.head(20)).mark_bar().encode(
    x=alt.X("Counts:Q", title="Frequency"),
    y=alt.Y("Start Word:N", sort="-x", title="Top Start Words"),
    tooltip=["Start Word", "Counts"]
).properties(
    title="Top 20 Start Words Frequency in Prompt",
    width=600,
    height=400
)

start_word_chart.show()

# TF-IDF Vectorization
stop_words = ["the", "is", "in", "and", "to", "a", "of", "for", "on", "with", "this", "that", "it", "as", "at", "an"]  # Ensure stop words are in list format
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=50)
X_tfidf = tfidf_vectorizer.fit_transform(df["PromptText"])
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# Get top words and their counts from TF-IDF
tfidf_sums = X_tfidf.sum(axis=0).A1  # Sum each column (word frequency across all prompts)
top_words_df = pd.DataFrame({"Top Keywords": feature_names, "Counts": tfidf_sums}).sort_values(by="Counts", ascending=False)


# Create bar chart using Altair for TF-IDF words
chart = alt.Chart(top_words_df.head(20)).mark_bar().encode(
    x=alt.X("Counts:Q", title="TF-IDF Counts"),
    y=alt.Y("Top Keywords:N", sort="-x", title="Top Keywords"),
    tooltip=["Top Keywords", "Counts"]
).properties(
    title="Top 20 Keywords in Prompt ",
    width=600,
    height=400
)

chart.show()



alt.Chart(...)

alt.Chart(...)

**From:Top 20 Start Words Frequency in Prompt**     
We could see the Key Patterns in Developer Prompts   
Question-Based Queries – Most prompts start with "how," "what," or "can," indicating a focus on explanations and best practices.   
Debugging & Errors – Terms like "error," "traceback" suggest frequent troubleshooting requests.   
Action-Oriented Requests – Words like "write," "find," "generate" highlight code generation expectations.   
Programming Topics – High TF-IDF keywords include "code," "data," "json," reflecting coding and API interests.   
Iterative Queries – Words like "now," "ok," indicate multi-step problem-solving.   

The top TF-IDF keywords analysis indicates frequent developer queries. High-ranking words like "how", "can", and "what" suggest users are looking for guidance or explanations. Technical terms such as "code", "data", "json", and "model" reflect strong interests in coding, data handling, and machine learning topics. Keywords like "error" and "file" highlight debugging and file management concerns, while terms like "python3" and "module" suggest Python's dominance in the queries. Visualizing the top 20 keywords with their importance helps prioritize areas like debugging, data processing, and library usage for targeted improvements.

In [33]:
# Count occurrences of each State value
state_chart = alt.Chart(state_df).mark_bar().encode(
    x=alt.X("Counts:Q", title="Frequency"),
    y=alt.Y("State:N", sort="-x", title="States"),
    tooltip=["State", "Counts"]
).properties(
    title="State Distribution",
    width=600,
    height=400
)

state_chart.show()

alt.Chart(...)

In [36]:
# Get State distribution for specific start words
target_words = ["how", "can", "you", "what", "be", "are"]
target_df = df[df["FirstWord"].isin(target_words)]
target_state_counts = target_df["State"].value_counts().reset_index()
target_state_counts.columns = ["State", "Counts"]

# Create bar chart using Altair for selected words' state distribution
target_state_chart = alt.Chart(target_state_counts).mark_bar().encode(
    x=alt.X("Counts:Q", title="Frequency"),
    y=alt.Y("State:N", sort="-x", title="State Distribution for Selected Start Words"),
    tooltip=["State", "Counts"]
).properties(
    title="State Distribution for 'How', 'Can', 'You', 'What', 'Be', 'Are'",
    width=600,
    height=400
)

target_state_chart.show()



alt.Chart(...)

No trend in these patterns correlate with the success of issue resolution were found